# Preprocesamiento

In [1]:
import pandas as pd

In [2]:
dict_mes = {'ene':'01',
            'feb':'02',
            'mar':'03',
            'abr':'04',
            'may':'05',
            'jun':'06',
            'jul':'07',
            'ago':'08',
            'sep':'09',
            'oct':'10',
            'nov':'11',
            'dic':'12'
        }

In [3]:
df = pd.read_csv('../data/raw/Dataset_GCRNO.csv')

Utilizamos minúscula para los nombres de cada característica, cambiamos el formato de los meses y año, y convertimos a formato datetime.

In [4]:
df.columns = map(str.lower, df.columns)

df['fecha'] = df['fecha'].replace(dict_mes, regex=True)
df['fecha'] = df['fecha'].apply(lambda x: x[0:6] + '20' + x[6:])
df['fecha']= pd.to_datetime(df['fecha'], format='%d-%m-%Y')

Extraemos dia, mes y año.

In [6]:
df['dia'] = pd.DatetimeIndex(df['fecha']).day
df['mes'] = pd.DatetimeIndex(df['fecha']).month
df['anio'] = pd.DatetimeIndex(df['fecha']).year

Obtenemos la columna demanda_energia.

In [7]:
columnas_horas = [f'dem_gcrno_h{i}' for i in range(24)]

df_long = df.melt(id_vars='fecha', value_vars=columnas_horas,
                  var_name='hora', value_name='demanda_energia')

df_long['hora'] = df_long['hora'].str.extract('(\d+)').astype(int)
df_long = df_long.sort_values(by=['fecha', 'hora']).reset_index(drop=True)

df_combined = df_long.merge(df, on=['fecha'])

df_combined = df_combined.drop(columns=columnas_horas)

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\andre\AppData\Local\Temp\ipykernel_33088\1431015579.py:6: SyntaxWarning: invalid escape sequence '\d'
  df_long['hora'] = df_long['hora'].str.extract('(\d+)').astype(int)


Eliminamos la columna fecha y ordenamos las columnas.

In [8]:
df_combined.drop(columns=['fecha'], inplace = True)

columns = df_combined.columns
ordered_columns = list([columns[0]]) + list(columns[-3:]) + list(columns[2:26]) + list([columns[1]])
df_final = df_combined[ordered_columns]

df_final.head()

,hora,dia,mes,anio,tmax-cab,tmax-hmo,tmax-obr,tmax-lmo,tmax-cul,tmin-cab,...,martes_postfestivo,semana_santa,1_mayo,10_mayo,16_sep,2_nov.,pre-navidad_y_new_year,navidad_y_new_year,post-navidad_y_new_year,demanda_energia
0,0,1,1,2007,21.0,22.0,25.0,30.0,29.0,2.0,...,0,0,0,0,0,0,0,1,0,1394
1,1,1,1,2007,21.0,22.0,25.0,30.0,29.0,2.0,...,0,0,0,0,0,0,0,1,0,1297
2,2,1,1,2007,21.0,22.0,25.0,30.0,29.0,2.0,...,0,0,0,0,0,0,0,1,0,1255
3,3,1,1,2007,21.0,22.0,25.0,30.0,29.0,2.0,...,0,0,0,0,0,0,0,1,0,1222
4,4,1,1,2007,21.0,22.0,25.0,30.0,29.0,2.0,...,0,0,0,0,0,0,0,1,0,1168
